In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("จาก raw data สู่ Disk").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        getOrCreate()

23/01/28 16:17:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Read the access log file
log_df = spark.read.text("access.log").withColumnRenamed('value','log_text')

In [3]:
! wc -l access.log

10365152 access.log


In [4]:
log_df.count()

10365152

In [5]:
### หลังจากบรรทัดนี้เป็นฝีมือ ChatGPT-3

In [6]:
from pyspark.sql.functions import regexp_extract

In [7]:
# Extract feature columns using regular expressions
log_df = log_df.withColumn("ip", regexp_extract("log_text", r"^([\d.]+)", 1))
log_df = log_df.withColumn("request", regexp_extract("log_text", r"\"(.*?)\"", 1))
log_df = log_df.withColumn("status", regexp_extract("log_text", r"\"\s+(\d+)", 1))
log_df = log_df.withColumn("size", regexp_extract("log_text", r"\"\s+\d+\s+(\d+)", 1))
log_df = log_df.withColumn("timestamp", regexp_extract("log_text", r"\[(.*?)\]", 1))
log_df = log_df.withColumn("user_agent", regexp_extract("log_text", r"\"(.*?)\"$", 1))


In [8]:
log_df.show()

+--------------------+-------------+--------------------+------+-----+--------------------+--------------------+
|            log_text|           ip|             request|status| size|           timestamp|          user_agent|
+--------------------+-------------+--------------------+------+-----+--------------------+--------------------+
|54.36.149.41 - - ...| 54.36.149.41|GET /filter/27|13...|   200|30577|22/Jan/2019:03:56...|GET /filter/27|13...|
|31.56.96.51 - - [...|  31.56.96.51|GET /image/60844/...|   200| 5667|22/Jan/2019:03:56...|GET /image/60844/...|
|31.56.96.51 - - [...|  31.56.96.51|GET /image/61474/...|   200| 5379|22/Jan/2019:03:56...|GET /image/61474/...|
|40.77.167.129 - -...|40.77.167.129|GET /image/14925/...|   200| 1696|22/Jan/2019:03:56...|GET /image/14925/...|
|91.99.72.15 - - [...|  91.99.72.15|GET /product/3189...|   200|41483|22/Jan/2019:03:56...|GET /product/3189...|
|40.77.167.129 - -...|40.77.167.129|GET /image/23488/...|   200| 2654|22/Jan/2019:03:56...|GET /